In [1]:
import urdf2casadi.urdfparser as u2c
import numpy as np

## 1. Load robot from urdf 

1. Create urdfparser-class instance. 
2. Load model to instance, either from file, string or ros parameter server. Examples uses from file. 

In [2]:
ur5 = u2c.URDFparser()
ur5.from_file("/home/lillmaria/urdf2casadi/examples/urdf/ur5_mod.urdf")

/home/lillmaria/urdf2casadi/examples/urdf/ur5_mod.urdf


## 2. Get joint information ? 
Information about the joints of the robot model can be obtained using u2c's get_joint_info(). "root" and "tip" are the inputs, and represent the link names for the root and tip of the kinematic tree one wishes to evaluate. 

In [3]:
root = "base_link"
tip = "tool0"

joint_list, joint_names, q_max, q_min = ur5.get_joint_info(root, tip)
n_joints = ur5.get_n_joints(root, tip)
print "name of first joint:", joint_names[0], "\n"
print "joint information for first joint:\n", joint_list[0]
print "\n q max:", q_max
print "\n q min:", q_min

name of first joint: shoulder_pan_joint 

joint information for first joint:
axis:
- 0.0
- 0.0
- 1.0
calibration: None
child: shoulder_link
dynamics:
  damping: 0.0
  friction: 0.0
limit:
  effort: 150.0
  lower: -6.28318530718
  upper: 6.28318530718
  velocity: 3.15
mimic: None
name: shoulder_pan_joint
origin:
  rpy:
  - 0.0
  - 0.0
  - 0.0
  xyz:
  - 0.0
  - 0.0
  - 0.089159
parent: base_link
safety_controller: None
type: revolute

 q max: [6.28318530718, 6.28318530718, 3.14159265359, 6.28318530718, 6.28318530718, 6.28318530718]

 q min: [-6.28318530718, -6.28318530718, -3.14159265359, -6.28318530718, -6.28318530718, -6.28318530718]


# 3. Obtain robot dynamics! 
After loading the robot model to the urdfparser-instance, the robot dynamics can be obtained from any given root and tip of the robot. 

## Dynamic parameters:
To obtain the dynamic parameters (M, C, G) of the equation of motion tau = M(q)q_ddot + C(q, q_dot)q_dot + G(q):

In [4]:
M_sym = ur5.get_inertia_matrix_crba(root, tip)
C_sym = ur5.get_coriolis_rnea(root, tip)

gravity = [0, 0, -9.81]
G_sym = ur5.get_gravity_rnea(root, tip, gravity)

M_sym, C_sym, G_sym are CasADi symbolic expressions of the ur5's dynamic parameters from given root to tip. The CasADi expressions are C-code generated so they can be numerically evaluated efficiently. Example of numerical evaluation is shown below. 

In [5]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    
M_num = M_sym(q)
C_num = C_sym(q, q_dot)
G_num = G_sym(q)
print "Numerical Inertia Matrx for random input: \n", M_num 
print "\nNumerical Coriolis term for random input: \n", C_num 
print "\nNumerical gravity term for random input: \n", G_num

Numerical Inertia Matrx for random input: 

[[3.04094, 0.260012, 0.0684082, -0.0014879, 0.194537, 0.00949564], 
 [0.260012, 3.95449, 1.51127, 0.247693, 0.010148, 0.0491895], 
 [0.0684082, 1.51127, 0.872107, 0.261585, 0.00410054, 0.0491895], 
 [-0.0014879, 0.247693, 0.261585, 0.275625, -0.00172613, 0.0491895], 
 [0.194537, 0.010148, 0.00410054, -0.00172613, 0.271565, 8.14897e-14], 
 [0.00949564, 0.0491895, 0.0491895, 0.0491895, 8.14897e-14, 0.0509585]]

Numerical Coriolis term for random input: 
[93.9265, 10.3752, 16.487, -5.47551, -7.60884, -1.31306]

Numerical gravity term for random input: 
[-8.88178e-16, -49.7737, -10.2919, 0.221171, -0.104471, 0]


## Inverse Dynamics 

Without accounting for gravitational forces:

In [6]:
tau_sym = ur5.get_inverse_dynamics_rnea(root, tip)

Accounting for gravitational forces:

In [7]:
gravity = [0, 0, -9.81]
tau_g_sym = ur5.get_inverse_dynamics_rnea(root, tip, gravity = gravity)

External forces can also be accounted for:

In [8]:
q = [None]*n_joints
q_dot = [None]*n_joints
q_ddot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_ddot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

tau_num = tau_sym(q, q_dot, q_ddot)
tau_g_num = tau_g_sym(q, q_dot, q_ddot)
#tau_fext_num = tau_fext_sym(q, q_dot, q_ddot)
print "Numerical inverse dynamics: \n", tau_num 
print "\nNumerical inverse dynamics w/ gravity: \n", tau_g_num 
#print "\nNumerical inverse dynamics w/ external forces: \n", G_num

Numerical inverse dynamics: 
[-9.12549, -3.59507, -9.66454, -4.9924, -4.84641, -0.435851]

Numerical inverse dynamics w/ gravity: 
[-9.12549, -7.16588, 4.9021, -5.02543, -4.83076, -0.435851]


## Forward Dynamics

urdf2casadi provides two methods for finding the robot's forward dynamics. The first method combines the recursive Newton-Euler algorithm (RNEA) and the composite rigid body algorithm (CRBA) and solves the equation of motion for the joint accelerations. The second method uses the articulated body algorithm (ABA) for forward dynamics. The method that uses ABA is in most cases the most efficient with regard to numerical evaluation, especially if the number of joints are high. (See timing examples for more information.)

In [9]:
tau = np.zeros(n_joints)
qddot_sym = ur5.get_forward_dynamics_crba(root, tip)

In [10]:
qddot_g_sym = ur5.get_forward_dynamics_aba(root, tip, gravity = gravity)


In [11]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

qddot_num = qddot_sym(q, q_dot, tau)
qddot_g_num = qddot_g_sym(q, q_dot, tau)

print "Numerical inverse dynamics: \n", qddot_num 
print "\nNumerical inverse dynamics w/ gravity: \n", qddot_g_num 

Numerical inverse dynamics: 
[1.31208, -13.6941, 44.2009, -27.2257, 0.614607, 12.1596]

Numerical inverse dynamics w/ gravity: 
[-0.0471928, -6.70821, 57.213, -47.5301, -0.722888, 12.4145]


# 4. Obtain the Derivatives 

From the dynamics functions, their derivatives can easily be obtained using CasADi`s built-in Jacobian functionality. The user can choose to find the derivative with regard to those variables needed (cs.jacobian()), or to find the time derivative with regard to these variables (cs.jtimes()), i.e the Jacobian times the time derivative of the variables.

If one are to find the time derivative, jtimes() is recommended over first obtaining the Jacobian, as jtimes() shortens the expressions, thus making the evaluation time of the expressions more efficient. 

Examples for obtaining the derivatives for the inverse dynamics, using both jacobian() and jtimes() are shown below:

## cs.jacobian()

The following explains how to use CasADi to obtain the derivative of the inverse dynamics with respect to q, qdot, and qddot, using the symbolic function returned by urdf2casadi (tau_sym):


1. Import CasADi and declare the symbolic variables needed in the derivative expression. 

In [12]:
import casadi as cs

q_sym =cs.SX.sym("qs", n_joints)
qdot_sym =cs.SX.sym("qsdot", n_joints)
qddot_sym =cs.SX.sym("qsddot", n_joints)

2. Declare the vector of the variables to find the derivatives with respect to using cs.vertcat():

In [13]:
id_vec = cs.vertcat(q_sym, qdot_sym, qddot_sym)
print id_vec

[qs_0, qs_1, qs_2, qs_3, qs_4, qs_5, qsdot_0, qsdot_1, qsdot_2, qsdot_3, qsdot_4, qsdot_5, qsddot_0, qsddot_1, qsddot_2, qsddot_3, qsddot_4, qsddot_5]


3. Obtain the symbolic expression of the derivative of ID with respect to q, qdot, and qddot using cs.jacobian():

In [14]:
derivative_id_expression = cs.jacobian(tau_sym(q_sym, qdot_sym, qddot_sym), id_vec)

4. Use the symbolic expression to make a CasADi function that can be efficiently numerical evaluated:

In [15]:
derivative_id_function  = cs.Function("did", [q_sym, qdot_sym, qddot_sym], [derivative_id_expression], {"jit": True, "jit_options":{"flags":"-Ofast"}})

where -Ofast flag is used to C-code generate the function. The derivative function can then be numerically evaluated similar to the functions returned by urdf2casadi, as illustrated in the above. For instance:

In [16]:
print derivative_id_function(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))


[[00, 3.47485, 0.783458, -1.88474, 0.103683, -0.174549, -1.41623, -0.896139, -0.234738, 0.128601, 0.0810386, 0.0776548, 0.986864, 0.436166, 0.0821034, 0.00166266, 0.258027, -0.00605124], 
 [00, -1.88078, -3.49342, 0.548254, -0.156966, -0.0290333, 1.18932, -1.20679, -2.4005, -0.0553704, -0.10926, -0.0637559, 0.436166, 3.42078, 1.24872, 0.249093, 0.0115794, 0.027533], 
 [00, -0.661822, -0.358749, 0.406655, -0.115436, -0.0290333, 0.0825389, 1.18105, -0.0126542, -0.0599878, -0.0810284, -0.0637559, 0.0821034, 1.24872, 0.88072, 0.26514, 0.00835043, 0.027533], 
 [00, 0.465165, 0.438444, 0.366942, -0.0587761, -0.0290333, -0.12426, 0.0522804, 0.0548047, 0.00747109, -0.0737805, -0.0637559, 0.00166266, 0.249093, 0.26514, 0.274123, 0.00559261, 0.027533], 
 [00, -0.769908, -0.775828, -0.75445, 0.11017, -0.143756, 0.0993527, 0.0872177, 0.0749549, 0.0727555, -0.0151721, 0.0842753, 0.258027, 0.0115794, 0.00835043, 0.00559261, 0.259837, 4.41439e-14], 
 [00, 0.108386, 0.108386, 0.108386, -0.165816, 0.1


One can also find the derivative with respect to just one variable:


In [17]:
derivative_id_expression_dq = cs.jacobian(tau_sym(q_sym, qdot_sym, qddot_sym), q_sym)
derivative_id_function_dq  = cs.Function("didq", [q_sym, qdot_sym, qddot_sym], [derivative_id_expression_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})

print derivative_id_function_dq(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))


[[00, 3.47485, 0.783458, -1.88474, 0.103683, -0.174549], 
 [00, -1.88078, -3.49342, 0.548254, -0.156966, -0.0290333], 
 [00, -0.661822, -0.358749, 0.406655, -0.115436, -0.0290333], 
 [00, 0.465165, 0.438444, 0.366942, -0.0587761, -0.0290333], 
 [00, -0.769908, -0.775828, -0.75445, 0.11017, -0.143756], 
 [00, 0.108386, 0.108386, 0.108386, -0.165816, 0.176506]]


# cs.jtimes()

To obtain the time derivative with cs.jtimes, the same procedure as for cs.jacobian() is used with an additional variable, i.e the time derivatives of the varibales:

1. Import casadi and declare the symbolic variables needed, also the time derivatives of these: 

In [18]:
import casadi as cs

q_sym =cs.SX.sym("qs", n_joints)
qdot_sym =cs.SX.sym("qsdot", n_joints)
qddot_sym =cs.SX.sym("qsddot", n_joints)
qdddot_sym = cs.SX.sym("qsdddot", n_joints)

2. Declare the vector of the variables to find the derivatives with respect to, and the vector with their time derivatives:

In [19]:
id_vec = cs.vertcat(q_sym, qdot_sym, qddot_sym)
id_dvec = cs.vertcat(qdot_sym, qddot_sym, qdddot_sym)

3. Obtain the symbolic expression of the time derivative of ID with respect to q, qdot, and qddot using cs.jtimes():

In [20]:
timederivative_id_expression = cs.jtimes(tau_sym(q_sym, qdot_sym, qddot_sym), id_vec, id_dvec)

4. Use the symbolic expression to make a CasADi function that can be efficiently numerical evaluated:

In [21]:
timederivative_id_function = cs.Function("didtimes", [q_sym, qdot_sym, qddot_sym, qdddot_sym], [timederivative_id_expression], {"jit": True, "jit_options":{"flags":"-Ofast"}})
print timederivative_id_function(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))

[1.80166, -2.26444, 2.80034, 1.85865, -1.387, 0.482345]


And one can also just find the time derivative of the inverse dynamics with respect to, for instance, q:

In [22]:
timederivative_id_expression_dq = cs.jtimes(tau_sym(q_sym, qdot_sym, qddot_sym), q_sym, qdot_sym)
timederivative_id_function_dq = cs.Function("dqidtimes", [q_sym, qdot_sym, qddot_sym], [timederivative_id_expression_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
print timederivative_id_function_dq(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))

[2.3027, -5.01195, -0.758384, 1.18274, -2.33377, 0.335847]
